In [24]:
import json
import numpy as np
import os
import pickle
import shutil
import sqlite3
from collections import defaultdict
from datetime import datetime
from time import sleep
#import query
import logzero
import requests
#import secret
import warnings
from pandas.core.common import SettingWithCopyWarning
from logzero import logger
from pandas.tseries.offsets import MonthEnd
from requests import get
import calendar
import regex as re
import pandas as pd

In [25]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [26]:
query_zip = """
SELECT *

FROM program

LEFT JOIN zipcode on program.state_id == zipcode.state_id

WHERE is_entire_state == 1
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

state_df = pd.read_sql_query(query_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [27]:
state_df.shape

(1677080, 30)

In [28]:
query_city_zip = """
SELECT *

FROM program_city

LEFT JOIN program

on program_city.program_id == program.id

LEFT JOIN zipcode on program_city.city_id == zipcode.city_id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

city_df = pd.read_sql_query(query_city_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [29]:
city_df.shape

(14048, 32)

In [30]:
query_county_zip = """
SELECT *

FROM program_county

LEFT JOIN program

on program_county.program_id == program.id

LEFT JOIN zipcode on program_county.county_id == zipcode.county_id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

county_df = pd.read_sql_query(query_county_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [31]:
county_df.shape

(4889, 32)

In [32]:
query_utility_zip = """
SELECT *

FROM program_utility

LEFT JOIN program

on program_utility.program_id == program.id

LEFT JOIN utility_zipcode

on program_utility.utility_id == utility_zipcode.utility_id

LEFT JOIN zipcode on utility_zipcode.zipcode_id == zipcode.id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

utility_df = pd.read_sql_query(query_utility_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [33]:
utility_df.shape

(434875, 34)

In [34]:
query_zip_zip = """
SELECT *

FROM program_zipcode

LEFT JOIN program

on program_zipcode.program_id == program.id

INNER JOIN zipcode on program_zipcode.zipcode_id == zipcode.id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

zip_df = pd.read_sql_query(query_zip_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [35]:
zip_df.shape

(36, 32)

In [36]:
county_df = county_df.loc[:,~county_df.columns.duplicated()]
city_df = city_df.loc[:,~city_df.columns.duplicated()]
state_df = state_df.loc[:,~state_df.columns.duplicated()]
zip_df = zip_df.loc[:,~zip_df.columns.duplicated()]
utility_df = utility_df.loc[:,~utility_df.columns.duplicated()]

keep_cols = list(
    set(zip_df.columns).intersection(
    set(utility_df.columns)).intersection(
    set(county_df.columns)).intersection(
    set(city_df.columns)).intersection(
    set(state_df.columns))
)



dsire_by_zip = pd.concat([zip_df[keep_cols],
                          utility_df[keep_cols],
                          county_df[keep_cols],
                          city_df[keep_cols],
                          state_df[keep_cols]])

In [37]:
query_tech = """
SELECT * 

FROM program_technology

WHERE program_technology.technology_id == 7
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

tech_df = pd.read_sql_query(query_tech, conn_proj)

conn_proj.close

<function Connection.close>

In [38]:
check_list = tech_df.program_id.to_list()

In [39]:
dsire_PV = dsire_by_zip[dsire_by_zip.id.isin(check_list)]

In [40]:
dsire_PV['start_date'] = dsire_PV.start_date.fillna(dsire_PV.created_ts)
dsire_PV['end_date'] = dsire_PV.end_date.fillna(datetime.strptime(cfg["max_date"], "%d-%b-%Y (%H:%M:%S.%f)"))
diff_df = dsire_PV.dropna(subset = ['start_date', 'zipcode']).reset_index()
diff_df['zipcode'] = diff_df.zipcode.apply(lambda n: str(int(n)).zfill(5))
diff_df.head()

,index,implementing_sector_id,id,fromSir,zipcode,longitude,start_date_text,end_date,websiteurl,fundingsource,...,state_id,administrator,program_category_id,city_id,summary,program_type_id,name,is_entire_state,start_date,published
0,0,3,9,0,80002,-105.0,None,2019-12-31 00:00:00,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,...,7,None,2,25521.0,Xcel Energy offers a net-metering program for ...,37,Xcel Energy - Net Metering,0,2003-09-05 00:00:00,0
1,1,3,9,0,80003,-105.0,None,2019-12-31 00:00:00,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,...,7,None,2,25521.0,Xcel Energy offers a net-metering program for ...,37,Xcel Energy - Net Metering,0,2003-09-05 00:00:00,0
2,2,3,9,0,80004,-105.0,None,2019-12-31 00:00:00,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,...,7,None,2,25521.0,Xcel Energy offers a net-metering program for ...,37,Xcel Energy - Net Metering,0,2003-09-05 00:00:00,0
3,3,3,9,0,80005,-105.0,None,2019-12-31 00:00:00,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,...,7,None,2,25521.0,Xcel Energy offers a net-metering program for ...,37,Xcel Energy - Net Metering,0,2003-09-05 00:00:00,0
4,4,3,9,0,80007,-105.0,None,2019-12-31 00:00:00,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,...,7,None,2,25521.0,Xcel Energy offers a net-metering program for ...,37,Xcel Energy - Net Metering,0,2003-09-05 00:00:00,0


In [41]:
query_sector = """
SELECT * 

FROM program_sector

"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

sector_df = pd.read_sql_query(query_sector, conn_proj)

conn_proj.close

<function Connection.close>

In [42]:
diff_df = diff_df.merge(sector_df, left_on = 'id', right_on = 'program_id', how = 'left')

In [46]:
diff_df = diff_df[diff_df.sector_id.isin([9,23])]
diff_df.sample(5)

,index,implementing_sector_id,id,fromSir,zipcode,longitude,start_date_text,end_date,websiteurl,fundingsource,...,program_category_id,city_id,summary,program_type_id,name,is_entire_state,start_date,published,program_id,sector_id
131376,29279,1,936,0,84640,-112.0,None,2019-12-31 00:00:00,http://www.energytrust.org,Public Benefits Fund,...,1,26526.0,<p><span>Energy Trust of Oregon’s Solar Electr...,88,Solar Electric Incentive Program,0,2003-05-01 04:00:00,1,936.0,23.0
2478715,841693,1,3554,0,43266,-83.0,10/16/2009,2019-12-31 00:00:00,None,None,...,1,12771.0,<p>&#10;&#9;<span><b><i><span>Note: </span></i...,76,Local Option - Special Energy Improvement Dist...,1,2009-10-16 04:00:00,1,3554.0,9.0
156644,32232,1,1087,0,06091,-73.0,10/1/2004,2012-03-05 00:00:00,http://www.ctcleanenergy.com/YourHome/SolarReb...,Clean Energy Finance and Investment Authority,...,1,1753.0,"<p><span style=""font-style: italic;""><span sty...",88,CEFIA - Residential Solar PV Rebate Program,0,2004-10-01 00:00:00,0,1087.0,9.0
2936656,1145080,1,5067,0,61283,-90.0,None,2019-12-31 00:00:00,http://www.icc.illinois.gov/electricity/Electr...,None,...,1,19418.0,"<p>&#10;&#9;In August 2007, Illinois enacted l...",13,Solar Renewable Energy Credits,1,2011-12-19 00:00:00,0,5067.0,9.0
2928572,1140369,1,5028,0,22245,-77.0,06/01/2012,2016-06-01 00:00:00,https://payments.vi.virginia.gov/SolarFund/,Donations from customers of investor-owned uti...,...,1,6515.0,<p><em><strong>DMME is currently developing cr...,89,Virginia Solar Resource Development Fund,1,2012-06-01 00:00:00,0,5028.0,9.0


In [18]:
#datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)")
diff_df.columns

Index(['index', 'implementing_sector_id', 'id', 'fromSir', 'zipcode',
       'longitude', 'start_date_text', 'end_date', 'websiteurl',
       'fundingsource', 'budget', 'code', 'created_by_user_id',
       'additional_technologies', 'end_date_text', 'created_ts', 'updated_ts',
       'county_id', 'latitude', 'state_id', 'administrator',
       'program_category_id', 'city_id', 'summary', 'program_type_id', 'name',
       'is_entire_state', 'start_date', 'published'],
      dtype='object')

In [47]:
program_map_df = diff_df[['program_type_id','zipcode','start_date','end_date']]
program_map_df['s_d'] = pd.to_datetime(program_map_df.start_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))
program_map_df['e_d'] = pd.to_datetime(program_map_df.end_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))

program_map_df['start_month'] = pd.to_datetime(program_map_df.start_date).dt.strftime('%Y-%m')
program_map_df['start_month'] = pd.to_datetime(program_map_df['start_month']) + MonthEnd(1)



In [48]:
program_map_df['nb_months'] = program_map_df['start_month'].apply(lambda x: ((x - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))/np.timedelta64(1, 'M')))
program_map_df['nb_months'] = program_map_df['nb_months'].astype(int)

In [49]:
program_map_df["s_d"] = program_map_df.s_d.dt.days
program_map_df["e_d"] = program_map_df.e_d.dt.days
program_map_df.head()

,program_type_id,zipcode,start_date,end_date,s_d,e_d,start_month,nb_months
1,37,80002,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67
3,37,80003,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67
5,37,80004,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67
7,37,80005,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67
9,37,80007,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67


In [50]:
def keys_exists(element, *keys):
    '''
    Check if *keys (nested) exists in `element` (dict).
    '''
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
        except KeyError:
            return False
    return True




program_map = defaultdict(dict)
    
for a, b, c, d in zip(program_map_df.program_type_id,program_map_df.zipcode,program_map_df.s_d,program_map_df.nb_months):
    
    if keys_exists(program_map, a, b):
        program_map[a][b].append([c,d])
    else:
        program_map[a][b] = [[c,d]]

In [51]:
with open('../data/program_zip_dates.p', 'wb') as fp:
    pickle.dump(program_map, fp, protocol=pickle.HIGHEST_PROTOCOL)

notebook was appearing in different folder. below code should correct issues with files not being found. 

In [52]:
# import os
# os.chdir("../../solar-697/ru")
# cwd = os.getcwd()  # Get the current working directory (cwd)
# files = os.listdir(cwd)  # Get all the files in that directory
# print("Files in %r: %s" % (cwd, files))

In [53]:
program_map_df.head()

,program_type_id,zipcode,start_date,end_date,s_d,e_d,start_month,nb_months
1,37,80002,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67
3,37,80003,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67
5,37,80004,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67
7,37,80005,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67
9,37,80007,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005,2003-09-30,67


In [54]:
program_map_df.start_month.min()

Timestamp('1975-01-31 00:00:00')

In [55]:
diff_df.columns

Index(['index', 'implementing_sector_id', 'id', 'fromSir', 'zipcode',
       'longitude', 'start_date_text', 'end_date', 'websiteurl',
       'fundingsource', 'budget', 'code', 'created_by_user_id',
       'additional_technologies', 'end_date_text', 'created_ts', 'updated_ts',
       'county_id', 'latitude', 'state_id', 'administrator',
       'program_category_id', 'city_id', 'summary', 'program_type_id', 'name',
       'is_entire_state', 'start_date', 'published', 'program_id',
       'sector_id'],
      dtype='object')

In [56]:
diff_df[['zipcode', 'summary', 'fundingsource','state_id', 'program_category_id', 'fromSir',
         'code', 'start_date', 'id', 'city_id', 'implementing_sector_id', 'websiteurl', 'budget',
         'program_type_id', 'county_id', 'name','end_date']]

,zipcode,summary,fundingsource,state_id,program_category_id,fromSir,code,start_date,id,city_id,implementing_sector_id,websiteurl,budget,program_type_id,county_id,name,end_date
1,80002,Xcel Energy offers a net-metering program for ...,None,7,2,0,CO20R,2003-09-05 00:00:00,9,25521.0,3,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,37,2814.0,Xcel Energy - Net Metering,2019-12-31 00:00:00
3,80003,Xcel Energy offers a net-metering program for ...,None,7,2,0,CO20R,2003-09-05 00:00:00,9,25521.0,3,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,37,2814.0,Xcel Energy - Net Metering,2019-12-31 00:00:00
5,80004,Xcel Energy offers a net-metering program for ...,None,7,2,0,CO20R,2003-09-05 00:00:00,9,25521.0,3,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,37,2814.0,Xcel Energy - Net Metering,2019-12-31 00:00:00
7,80005,Xcel Energy offers a net-metering program for ...,None,7,2,0,CO20R,2003-09-05 00:00:00,9,25521.0,3,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,37,2814.0,Xcel Energy - Net Metering,2019-12-31 00:00:00
9,80007,Xcel Energy offers a net-metering program for ...,None,7,2,0,CO20R,2003-09-05 00:00:00,9,25521.0,3,http://www.xcelenergy.com/docs/corpcomm/psco_e...,None,37,2814.0,Xcel Energy - Net Metering,2019-12-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3867668,04986,<p>Maine offers a property tax exemption for s...,None,23,1,0,ME29F,2020-04-01 04:00:00,22141,1403.0,1,https://www.maine.gov/revenue/taxes/tax-relief...,None,78,59.0,Solar and Wind Energy Property Tax Exemption,2019-12-31 00:00:00
3867673,04987,<p>Maine offers a property tax exemption for s...,None,23,1,0,ME29F,2020-04-01 04:00:00,22141,1404.0,1,https://www.maine.gov/revenue/taxes/tax-relief...,None,78,59.0,Solar and Wind Energy Property Tax Exemption,2019-12-31 00:00:00
3867678,04988,<p>Maine offers a property tax exemption for s...,None,23,1,0,ME29F,2020-04-01 04:00:00,22141,1405.0,1,https://www.maine.gov/revenue/taxes/tax-relief...,None,78,59.0,Solar and Wind Energy Property Tax Exemption,2019-12-31 00:00:00
3867683,04989,<p>Maine offers a property tax exemption for s...,None,23,1,0,ME29F,2020-04-01 04:00:00,22141,1406.0,1,https://www.maine.gov/revenue/taxes/tax-relief...,None,78,57.0,Solar and Wind Energy Property Tax Exemption,2019-12-31 00:00:00


In [58]:
diff_df[diff_df.program_type_id == 88].name.unique()

array(['Glendale Water and Power - Solar Solutions Program',
       'Long Island Power Authority - PV Rebate Program',
       'Klickitat PUD - Solar PV Rebate',
       'Silicon Valley Power - Solar Electric Buy Down Program',
       'NV Energy - Clean Energy Incentive Program',
       'Riverside Public Utilities - Residential PV Incentive Program',
       'Burbank Water and Power - Residential and Commercial Solar Support Program',
       'SMUD - PV Residential Retrofit Buy-Down',
       'Nevada Power - PV Rebate Program',
       'Ukiah Utilities - PV Buydown Program',
       'IID Energy - PV Solutions Rebate Program',
       'Holy Cross Energy - Renewable Energy Rebate Program',
       'Ashland Electric Utility - Photovoltaic Rebate Program',
       'Orcas Power & Light - Photovoltaic Rebate',
       'LADWP - Solar Incentive Program',
       'Pasadena Water and Power - Solar Power Installation Rebate',
       'Solar and Wind Energy Rebate Program',
       'OTEC - Residential Photovolt